### MainQuese02

#### 1. Character 클래스 만들기
- 이름, 레벨, 체력, 공격력, 방어력의 속성을 가짐
- 인스턴스의 현재 체력이 0 이상인지 bool 값을 반환하는 is_alive 메서드 만들기
- 공격을 받았을 때, (받은 데미지 - 본인의 방어력)만큼 현재 체력이 감소하는 take_damage 메서드 만들기
    - 본인의 방어력이 데미지보다 크다면 체력 감소하지 않음
- 타겟에게 데미지를 입히는 attack_target 메서드 만들기
    - 데미지는 1부터 공격력 사이의 랜덤한 정수

In [8]:
import random

class Character:
    def __init__(self, name, level, HP, damage, defence):
        self.name = name
        self.level = level
        self.HP = HP
        self.damage = damage
        self.defence = defence

    def is_alive(self):
        return self.HP > 0
    
    def take_damage(self, received_damage):
        actual_damage = max(0, received_damage - self.defence)
        self.HP -= actual_damage
        print(f"[{self.name}이(가) {actual_damage}만큼의 피해를 입었습니다. (남은 체력 : {self.HP})]")

    def attack_target(self, target):
        hit_damage = random.randint(1, self.damage)
        print(f"[{self.name}이(가) {target.name}에게 {hit_damage}의 위력으로 공격했습니다!]")
        target.take_damage(hit_damage)

#### 2. Player 클래스와 Monster 클래스 만들기 
- **Player 클래스**
    - Character를 상속 받기
    - 레벨 1, 체력 100, 공격력 25, 방어력 5로 초기화하기
    - Player 클래스는 경험치 속성을 추가로 가짐
    - 인수로 받은 정수 만큼 경험치를 획득하는 gain_experience 메서드 만들기
    - 현재 경험치가 50이상이면 레벨을 1, 공격력을 10, 방어력을 5씩 올리는 level_up 메서드 만들기
- **Monster 클래스**
    - Character를 상속 받기
    - 몬스터의 레벨에 비례하는 체력, 공격력, 방어력 초기화하기
        - 체력: 10~30 사이의 랜덤한 정수 * 레벨
        - 공격력: 5~20 사이의 랜덤한 정수 * 레벨
        - 방어력: 1~5 사이의 랜덤한 정수 * 레벨

In [9]:
class Player(Character):
    def __init__(self, name):
        super().__init__(name, 1, 100, 25, 5)
        self.exp = 0

    def gain_experience(self, amount):
        print(f"[{self.name}이(가) {amount}의 경험치를 얻었습니다. (현재 경험치 : {self.exp})]")

        if self.exp > 50:
            self.level_up()

    def level_up(self):
        while self.exp > 50:
            self.exp -= 50
            self.level += 1
            self.damage += 10
            self.defence += 5
            print(f"[LEVEL UP!! 레벨 {self.level}이 되었습니다.]")
            print(f"[공격력이 {self.damage}, 방어력이 {self.defence}(으)로 올라갔습니다.]")

In [10]:
class Monster(Character):
    def __init__(self, name, level):
        hp_per_level = random.randint(10, 30)
        damage_per_level = random.randint(5, 20)
        defence_per_level = random.randint(1, 5)

        calculated_hp = hp_per_level * level
        calculated_damage = damage_per_level * level
        calculated_defence = defence_per_level * level

        super().__init__(name, level, calculated_hp, calculated_damage, calculated_defence)
        
        print(f"[새로운 몬스터 {self.name} (레벨 {self.level})이(가) 나타났습니다!]")
        print(f"[체력: {self.HP}, 공격력: {self.damage}, 방어력: {self.defence}]")

#### 3. battle 함수 만들기
- **battle 함수**
    - Player 인스턴스와 Monster 인스턴스를 인수로 받아 둘 중 하나의 체력이 0 이하가 될 때까지 공격을 주고 받는 함수
    - 만약 Player 인스턴스가 살아남았다면
        - Player 인스턴스에 (몬스터 레벨 * 20)만큼의 경험치를 추가
        - player의 레벨업 메서드 호출
        - ‘전투 승리!’를 출력
    - Player 인스턴스가 살아남지 못했을 경우
        - ‘전투 패배..’를 출력하기

In [11]:
def battle(player, monster):
    print(f"\nLv.{monster.level}{monster.name}와(과) 전투 시작!")
    # 플레이어가 공격
    while player.is_alive() and monster.is_alive():
        playerattack_target(monster)
        if not monster.is_alive():
            break
        # 몬스터가 공격
        monster.attack_target(player)

    if player.is_alive():
        print(f"[{monster.name} 전투 승리!]")
        player.gain_experience(monster.level * 20)
        return True
    else:
        print("전투 패배...")
        return False

#### 4. main 함수 만들기
- 몬스터의 이름, 레벨이 매핑된 딕셔너리 정의하기
    - monster_dict = {‘슬라임’: 1, ‘고블린’: 2, ‘오크’: 3}
- 사용자로부터 이름을 입력받아 Player 인스턴스 생성하기
- 몬스터 딕셔너리로부터 Monster 인스턴스 생성하기
- 생성된 Monster 인스턴스와 Player 인스턴스가 battle 함수를 통해 전투
    - player는 생성된 몬스터 3마리(슬라임, 고블린, 오크)와 모두 전투해야함

전투 도중에 Player가 사망하면 이후 전투를 진행하지 않고 ‘게임오버’ 출력하고 종료

In [12]:
def main():
    monster_dict = {'슬라임': 1, '고블린': 2, '오크': 3}
    user_name = input("플레이어의 이름을 입력하세요")
    player = Player(user_name)

    print(f"\n게임 시작! {player.name}이(가) 전투를 시작합니다.")

    for m_name, m_level in monster_dict.items():
        monster = Monster(m_name, m_level)
        success = battle(player, monster)

        if not success:
            print("\n{player.name}이(가) 사망했습니다. 게임오버")
            return
        
    print(f"\n축하합니다! 모든 몬스터를 물리쳤습니다. {player.name}의 최종 레벨 : {player.level}")

if __name__ == "__main___":
    main()

        